# Description
This script's purpose is to create a table of datatype comma separated values(csv) from the image pixel values.


In [ ]:
# Required libraries
import requests
from urllib.parse import urlparse
from pathlib import Path
from datetime import datetime
import tifffile as tiff
import datetime
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [ ]:
def load_file(fp):
    """Takes a PosixPath object or string filepath
    and returns np array"""
    
    return tiff.imread(fp.__str__())

In [ ]:
bands = ['B01', 'B02', 'B03', 'B04', 'B05', 'B06', 'B07', 'B08', 'B8A', 'B09', 'B11', 'B12']

In [ ]:
# Sample file to load. Maintain the structure of where the images should be placed. The rest of the script will require this same arrangement
file_name = "data/00/20190314/0_B01_20190314.tif"
band_data = tiff.imread(file_name)
band_data

In [ ]:
#Take note of the filenames and file locations.
for tile in range(1):
    fids = f'./data/0{tile}/{tile}_field_id.tif'
    labs = f'./data/0{tile}/{tile}_label.tif' #only use this when preparing csv to train the model
    fid_arr = load_file(fids)
    lab_arr = load_file(labs)#only use this when preparing csv to train the model

In [ ]:
# NOTE: Code slice being replaced has col_loc starting at 1 and fid starting at 1
df = pd.DataFrame(
    list(np.ndindex((fid_arr.shape[0],
    fid_arr.shape[1]))), columns =['row_loc', 'col_loc']
)
df["fid"] = fid_arr.flatten().astype(int)
df["tile"] = 0
print(len(df))
print(df.head())

In [ ]:
print(len(df))
print(df.groupby('fid').count().shape)
print(df.columns)
df.head()

In [ ]:
tile = 0
d = datetime.datetime(2019,3,14)# Replace with the date or dates for the data you are using 

print(str(d))
t = '0' + str(tile)
for b in bands: # 3) For each band
    print(b,  d.strftime("%m%d%Y"))
    col_name = d.strftime("%Y%m%d") + '_' + b

    # if tile == 0:
    # If the column doesn't exist, create it and populate with 0s
    if col_name not in df.columns:
        df[col_name] = 0

#             Load im
    try:
        im = load_file(f"data/{t}/{d.strftime('%Y%m%d')}/{t[1]}_{b}_{d.strftime('%Y%m%d')}.tif")
        print(f"__________{im.shape}")
    except FileNotFoundError:
        continue

    # NOTE: Pixel values are added accoriding to tile position in original.
    # Since it's one tile, df.loc[df.tile == tile][['row_loc', 'col_loc']].values
    # will return all row_loc and col_loc ptns of the df. so im.flatten and assign can replace this
    df[col_name] = im.flatten().astype(int)

In [ ]:
df.head()

In [ ]:
df.to_csv('cashew_20190314.csv', index=False)